In [1]:
import math
from math import log, floor, ceil, sqrt

In [2]:
################################## CONSTANTS ###############################

# time to compute one AES hash (in nanoseconds)
aes_time = 10
gigabyte = 2 ** 30

In [3]:
# Calculates expected time of the main loop of PoST proof generation.
# The main loop is the sequential pass over the whole storage of the miner.
# 
# During the main loop the storage is processed as 16-bytes long labels. For every label we generate
# a pseudo-random binary sequence with a series of AES invocations. Every AES invocation produces 128 bits
# of output. This output is split into elements of equal binary size, which are the "eager" parts of the
# pseudorandom values we produce.
#
# Formally, we always compare 64-bit value (obtained via some pseudorandom generation) with a 64-bit threshold.
# But implementation-wise this comparison is optimized by computing only some number of most significant bits
# of this value first (the "eager" part) and the remaining bits only if needed (the "lazy" part).
#
# During this loop we attempt to find a specified number of "good" <label,nonce> pairs, where this pair is good
# if for a corresponding value v this formula holds:  v < threshold.
#
# The detailed description of the algorithm can be found here (ADD THE LINK).
#
# storage_size: int - size of miner's storage (in GiB)
# number_of_nonces: int - number of nonces that we want to calculate during the single-pass proof generation
# number_of_segments: int - we split the 128 bits of AES output to this number of segments
#                           caution: we require that 128 % number_of_segments == 0
def expected_time_of_sequential_pass(storage_size, number_of_nonces, number_of_segments):
    # ensuring that the splitting of AES output to segments is nice
    assert 128 % number_of_segments == 0

    # number of labels (every label is 16-bytes long)
    N = storage_size * gigabyte // 16

    # size of the unsigned integer values we will get after splitting the AES output
    segment_size_in_bits = 128 // number_of_segments
    
    # how many times we have to call AES per every label so to cover required number of nonces
    # this is just counting the "main" branch
    aes_calls_per_label = ceil(number_of_nonces / number_of_segments)
    
    # number of AES calls for the 'eager' part of values
    eager_part_calculations = N * aes_calls_per_label
    
    # average number of AES calls needed for the "lazy" part of values
    # Caution: only if the 'eager' part is EQUAL to some value, the lazy part must be calculated
    # hence the probability of the need to calculate the lazy part is 1 / 2 ** segment_size_in_bits.
    # The number of times we play the eager-lazy game is exactly: N * number_of_nonces   
    lazy_part_calculations = N * number_of_nonces // 2 ** segment_size_in_bits
    
    # total number of AES calls
    aes_calls_total = eager_part_calculations + lazy_part_calculations
    
    return aes_calls_total * aes_time

In [4]:
solution_eager64 = expected_time_of_sequential_pass(256, 32, 2)
solution_32_lazy32 = expected_time_of_sequential_pass(256, 32, 4)
solution_16_lazy48 = expected_time_of_sequential_pass(256, 32, 8)
solution_8_lazy56 = expected_time_of_sequential_pass(256, 32, 16)
solution_4_lazy60 = expected_time_of_sequential_pass(256, 32, 32)

In [5]:
print("    eager 64 [sec]: ", solution_eager64 / 10 ** 9)
print("32 + lazy 32 [sec]: ", solution_32_lazy32 / 10 ** 9)
print("16 + lazy 48 [sec]: ", solution_16_lazy48 / 10 ** 9)
print(" 8 + lazy 56 [sec]: ", solution_8_lazy56 / 10 ** 9)
print(" 4 + lazy 60 [sec]: " , solution_4_lazy60 / 10 ** 9)
print()
print("performance jump 64 -> 32: ", solution_eager64 / solution_32_lazy32)
print("performance jump 32 -> 16: ", solution_32_lazy32 / solution_16_lazy48)
print("performance jump 16 -> 8: ", solution_16_lazy48 / solution_8_lazy56)
print("performance jump 16 -> 8: ", solution_16_lazy48 / solution_8_lazy56)
print("performance jump 8 -> 4: ", solution_8_lazy56 / solution_4_lazy60)

    eager 64 [sec]:  2748.77906944
32 + lazy 32 [sec]:  1374.389536
16 + lazy 48 [sec]:  687.27865344
 8 + lazy 56 [sec]:  365.07222016
 4 + lazy 60 [sec]:  515.39607552

performance jump 64 -> 32:  1.9999999981373549
performance jump 32 -> 16:  1.9997558910361026
performance jump 16 -> 8:  1.8825827205882353
performance jump 16 -> 8:  1.8825827205882353
performance jump 8 -> 4:  0.7083333333333334


In [6]:
solution20nonces_32_lazy32 = expected_time_of_sequential_pass(256, 20, 4)
solution20nonces_16_lazy48 = expected_time_of_sequential_pass(256, 20, 8)
solution20nonces_8_lazy56 = expected_time_of_sequential_pass(256, 20, 16)
print("32 + lazy 32 [sec]: ", solution20nonces_32_lazy32 / 10 ** 9)
print("16 + lazy 48 [sec]: ", solution20nonces_16_lazy48 / 10 ** 9)
print(" 8 + lazy 56 [sec]: ", solution20nonces_8_lazy56 / 10 ** 9)

32 + lazy 32 [sec]:  858.99346
16 + lazy 48 [sec]:  515.44850432
 8 + lazy 56 [sec]:  357.01915648
